In [1]:
!pip install nltk
!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install numpy
!pip install sklearn
!pip install nltk

In [2]:
import nltk
import numpy as np
import random
import string

In [3]:
with open('data.txt','r',errors = 'ignore') as f:
  raw=f.read()

In [4]:
raw = raw.lower() 

In [5]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shker\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shker\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [7]:
sent_tokens[:5]

['во время моего недавнего визита в сша, я зашел в офис компании adobe в сан-франциско для того, чтобы по больше разузнать про apollo – новую платформу компании для разработки и выполнения ria (rich internet application).',
 'я пообщался с майком чемберсом – продакт-менеджером ответственным за apollo, чтобы узнать, что же такое на самом деле apollo, и каково его будущее.',
 'кристиан контрелл, другой менеджер в команде apollo, присоединился к нам в середине разговора, чтобы показать мне некоторые новые приложения сделанные с помощью apollo.',
 'в этом тексте я суммирую все то, что узнал об apollo, а так же добавлю краткое описание «для чайников» (потому что до этой встречи я был одним из этих «чайников»).',
 'почти все из того, что писалось уже об apollo в блогах и сми отражало преимущества apollo для разработчиков, я же хочу так же представить удобства apollo для конечного пользователя, так что этот текст являет собой некую попытку это сделать.']

In [8]:
word_tokens[:5]

['во', 'время', 'моего', 'недавнего', 'визита']

In [9]:
import pymorphy2

lemmer = pymorphy2.MorphAnalyzer()

In [10]:
# from nltk.stem.wordnet import WordNetLemmatizer

# eng_lemmer = WordNetLemmatizer()

In [11]:
# eng_lemmer.lemmatize('going')

In [12]:
def LemTokens(tokens):
    return [lemmer.parse(token)[0].normal_form for token in tokens]
    # return [eng_lemmer.lemmatize(token) for token in tokens]
    
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [27]:
import re 
from IPython.display import HTML

# Тут нужно менять!
GREETING_INPUTS = ("привет", "здравствуйте", "здорово", "чо каво", "хай", "салют", "хаюшки")
GREETING_RESPONSES = ["привет", "хай", "*подмигнул*", "здарова", "здравствуй", "рад общению с тобой"]
STRANGE_ITEM = ("абоба")

called=False

def greeting(sentence):
    global called
    if len(sentence.split()) > 0:
        sentence = re.sub('[^А-Яа-яё ]', '', sentence)
#         sentence = re.sub('[^A-Za-z ]', '', sentence)
    if sentence in STRANGE_ITEM and not called:
        display(HTML('<iframe allow="autoplay" width="560" height="315" src="https://www.youtube.com/embed/3HrSVXP99kQ?autoplay=1" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'))
        called = True
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

In [29]:
def response(user_response):
    robo_response=''
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words("russian"))
    # TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words="english")
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Извини, но я не понимаю тебя!" # ТУТ можно менять :) 
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [30]:
import warnings
warnings.filterwarnings('ignore')

flag=True

# Тут тоже нужно менять!
print("Робот: Да Да. \nМне лень писать сввоё имя, но если ты спросишь... \nХочешь мем введи абоба")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='пока'):
        if(user_response in ['благодарю', 'спс']): # И тут меняйте :)
            flag=False
            print("Робот: Иче??? ;)")
        else:
            if(greeting(user_response)!=None):
                print("Робот: "+greeting(user_response))
            else:
                sent_tokens.append(user_response)
                word_tokens=word_tokens+nltk.word_tokenize(user_response)
                final_words=list(set(word_tokens))
                if user_response == "как тебя зовут":
                    display(HTML('<iframe allow="autoplay" width="560" height="315" src="https://www.youtube.com/embed/W85F-UmnbF4?start=39&autoplay=1" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'))
                print("Робот: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("Робот: Пока-пока! *машет-ручкой*") # Тут тоже нужно импровизировать

Робот: Да Да. 
Мне лень писать сввоё имя, но если ты спросишь... 
Хочешь мем введи абоба
абоба


Робот: абоба
привет
Робот: здравствуй
кто ты 
Робот: Извини, но я не понимаю тебя!
apollo
Робот: apollo

второй наш кандидат – это apollo.
о чём речь
Робот: речь идёт о хуках usequery, usemutation и usesubscription.
библиотека
Робот: таким образом, мы выбрали библиотеку и пошли работать.
GraphQl
Робот: что такое graphql?
как тебя зовут


Робот: меня зовут oveveveevuev eyentanebwueve ugmbagbem osas!
пока
Робот: Пока-пока! *машет-ручкой*
